In [3]:
import numpy as np
import mdtraj as md
import matplotlib.pyplot as plt
import nglview as nv

# import pmcpy as mcpy 
import numpy as np
import matplotlib.pyplot as plt
#from pmcpy import so3, Chain, RBP, Pivot, Crankshaft, Dumps, SingleTriad, ClusterTrans
import pmcpy.run.equilibrate as mc

# # # Path: pymdna/__init__.py, prototype of the package and atomic is not properly referenced in the package at genertors.py now I just explicitly define the path loction
# # import pymdna as mdna
# import sys
# sys.path.append('/Users/thor/surfdrive/Projects/pymdna/')
# import pymdna as mdna 
# %load_ext autoreload
# %autoreload 2



ModuleNotFoundError: No module named 'pmcpy'

In [ ]:
np.set_printoptions(linewidth=250,precision=3,suppress=True)

npb  = 100
closed = False
conf = np.zeros((npb,4,4))
gs = np.array([0,0,0.6,0,0,0.34])
g = so3.se3_euler2rotmat(gs)
conf[0] = np.eye(4)
for i in range(1,npb):
    g = so3.se3_euler2rotmat(gs+np.random.normal(0,0.1,6))
    conf[i] = conf[i-1] @ g
seq = ''.join(['ATCG'[np.random.randint(4)] for i in range(npb)])
specs = {'method':'MD','gs_key': 'group_gs','stiff_key': 'group_stiff'}

ch = Chain(conf,keep_backup=True,closed=closed)
bps = RBP(ch,seq,specs,closed=closed,static_group=True)    

moves = list()
moves.append(Pivot(ch,bps))

for i in range(1000):
    for move in moves:
        move.mc()

moves = list()
moves.append(Crankshaft(ch,bps,2,50))
moves.append(SingleTriad(ch,bps))
moves.append(ClusterTrans(ch,bps,2,25))

Es = []
confs = []
confs.append(np.copy(ch.conf[:,:3,3]))

for i in range(100000):
    for move in moves:
        move.mc()
    
    if i%1000==0:
        print(f'step {i}: ')
        for move in moves:
            print(f'{move.name}: {move.acceptance_rate()}')
        Es.append(bps.get_total_energy())
        confs.append(np.copy(ch.conf[:,:3,3]))


print(f'<E> / DoFs = {np.mean(Es)/(len(ch.conf)-1)/6}')   

types = ['C' for i in range(ch.nbp)]
data = {'pos':confs, 'types':types}
Dumps.xyz.write_xyz('test.xyz',data)
